In [1]:
import random

In [161]:
class Side:
    def __init__(self, name, objs):
        self.name = name
        self.objs = objs
        self.first_time = True
    def move_to(self, other_side, n):

        # select obj to move
        random.shuffle(self.objs)
        if self.first_time and self.name=='right-side':
            self.first_time = False
            res = True
            moving = random.choice([[-8,8],[5,-5],[3,-3]])
            moving = [3,-3]
         
            for i in moving:
                obj = self.objs.remove(i)
                other_side.receive(i)
            action = self.cvt_moving_to_text(moving, other_side)
           
            return res, action
        
        n_failed = 0
        while True:
            moving = []
            if len(self.objs) <= 2:
                n = 1
            if self.name == 'right-side' and len(self.objs)==2:
                res, action = True, 'winning'
                break
                
            # find person
            for i in self.objs:
                if i < 0:
                    self.objs.remove(i)
                    moving.append(i)
                    other_side.receive(i)
                    break
            if n == 2:
                obj = self.objs.pop(0)
                moving.append(obj)
                other_side.receive(obj)

            # check valid
            res = self.check_valid(other_side, moving)
            if res: 
                break
            else:
                # return obj
                for i in moving:
                    self.objs.append(i)
                    other_side.objs.remove(i)
                random.shuffle(self.objs)
                n_failed += 1
                if n_failed > 30:
                    return False, 'failed'
     

        action = self.cvt_moving_to_text(moving, other_side)
        
        
        return res, action
    def print_(self, other_side):
        print(self.objs, other_side.objs)
        
    def check_valid(self, other_side, moving):
        found = False
        if self.name == 'left-side' and len(moving)==2:
            if moving[0]<0 and moving[1]<0:
                return False
        
        if self.is_valid() and other_side.is_valid():
            return  True
        else: 
            return False
    def cvt_moving_to_text(self, moving, other_side):
        text = 'move'
        for i in moving:
            text += ' ' + str(i)
        text += ' to ' + other_side.name +'\n'
        return text
    def receive(self, obj):
        self.objs.append(obj)
    def is_valid(self):
        my_map = {
            'red':-8,
            'green':-5,
            'blue':-3,
            '8':8,
            '5':5,
            '3':3
        }

        found_person = False
        for i in self.objs:
            if i < 0:
                found_person = True
        
        if found_person:
            sum_ = sum(self.objs)
            if sum_ > 0:
                return False

        return True
def run():
    left = Side('left-side', [])
    right = Side('right-side', [-8,-5,-3,8,5,3])
    cnt = 0
    moving_success = []
    while True:
        success, action = right.move_to(left, 2)
        if not success: 
            break
        moving_success.append(action)
        cnt += 1

        # check win
        print(len(left.objs), 'len left objs')
        if len(left.objs) == 6:
            return cnt, moving_success
        success, action = left.move_to(right, random.randint(1,2))
        if not success:
            break
        moving_success.append(action)
        cnt += 1
    return 9999, moving_success


In [115]:
left = Side('left-side', [])
right = Side('right-side', [-8,-5,-3,8,5,3])

In [108]:
right.move_to(left, 2)

right-side [-8, 8] [3, -3, -5, 5]


(True, 'move -5 5 to left-side\n')

In [110]:
left.move_to(right, random.randint(1,2))

KeyboardInterrupt: 

In [109]:
print(left.objs,right.objs)

[3, -3, -5, 5] [-8, 8]


In [531]:
def to_left():
    # <
    global boat, right, left
    while True:
        random.shuffle(right)
        pop_person(right)
        pop_random(right)
        res = ck_on_boat(left)
        if res: break
        else:
            # down boat
            for i in boat:
                right.append(i)
            boat = []
    print_from('right')
    move_to(left)
    
def to_right():
    # >
    global boat, left, right
    while True:
        random.shuffle(left)
        pop_person(left)
        res = ck_on_boat(right)
        if res:break
        else:
            # down boat
            for i in boat:
                left.append(i)
    print_from('left')
    move_to(right)
    
def to_right2():
    # >
    global boat, left, right
    while True:
        random.shuffle(left)
        pop_person(left)
        res = ck_on_boat(right)
        if res:break
        else:
            # down boat
            for i in boat:
                left.append(i)
    print_from('left')
    move_to(right)
    
def winning_step():
    global boat, left, right
    if len(right) == 1:
        person = -right[0]
        boat.append(person)
        left.remove(person)
        move_to(right)
        
        to_left()
        return True
    return False
    
            

In [557]:
def pop_person(side):
    global boat, left, right
    random.shuffle(side)
    for i in side:
        if i <0:
            side.remove(i)
            boat.append(i)
            break
    

def ck(side):
    has_person = False
    for i in side:
        if i <0:
            has_person = True
    if has_person:
        if sum(side)>0:
            return False
    return True

def pop_random(side):
    global boat, left, right
    while True:
        pop = side.pop(0)
        res = ck(side)
        if res:
            break
        else:
            side.append(pop)
            random.shuffle(side)
    boat.append(pop)
    return pop

def move_to(side):
    global boat, left, right
    for i in boat:
        side.append(i)
    boat = []
    

my_map = {
        '-8':'R ',
        '-5':'G ',
        '-3':'B ',
        '8':'8 ',
        '5':'5 ',
        '3':'3 ',
        'empty':'  '
    }
def convert_boat(boat):
    temp=boat.copy()
    temp.sort()
    ans = '['
    for i in temp:
        ans += my_map[str(i)]
    if len(temp)==1:
        ans += my_map['empty']
    ans += ']'
    return ans

def convert_to_txt(side):
    temp=side.copy()
    ans = ''
    
    temp.sort()
    for i in temp:
        ans += my_map[str(i)]
    for i in range(4-len(temp)):
        ans += my_map['empty']
    return ans
def write():
    pass
def print_from(side_name):
    global count, action
    count += 1
    if side_name == 'left':
        ans = ''
        ans += convert_to_txt(left)
        ans += '|'
        ans += convert_boat(boat)
        ans += '->         |'
        ans += convert_to_txt(right)
    else:
        ans = ''
        ans += convert_to_txt(left)
        ans += '|        <-'
        ans += convert_boat(boat)
        ans += ' |'
        ans += convert_to_txt(right)
    action += ans + '\n'
def ck_on_boat(to_side):
    global boat, left, right
    temp = []
    for i in to_side:
        temp.append(i)
    for i in boat:
        temp.append(i)
    return ck(temp)
def pop_money(side):
    global boat
    random.shuffle(side)
    for i in side:
        if i >0:
            side.remove(i)
            boat.append(i)
            break

In [572]:
left, right, boat = [],[],[]
solution = []
n_solution = 1
action = 'solution 1\n'
def solve():
    global left, right, boat
    left = []
    right = [-8,-5,-3,8,5,3]
    boat = []

    
    to_left()
    to_right()
    
    
    # <
    _ = 0
    while True:
        _ += 1
        if _ > 20:
            return False
        random.shuffle(right)
        pop_person(right)
        pop_person(right)
        res = ck_on_boat(left)
        if res: break
        else:
            # down boat
            for i in boat:
                right.append(i)
            boat = []
    print_from('right')
    move_to(left)
    
    
    # >
    while True:
        random.shuffle(left)
        pop_person(left)
#         ##### inverse
#         inv = -boat[0]
#         print(inv)
#         print(left, boat, right)
#         boat.append(inv)
#         left.remove(inv)
        
        res = ck_on_boat(right)
        if res:break
        else:
            # down boat
            for i in boat:
                left.append(i)
    print_from('left')
    move_to(right)
    
    
    
    to_left()
    to_right2()
    to_left()
    to_right()
    return True
    if count == 5:
        if len(left)==6 and len(right)==0:
            return True
        else:
            return False

        
for i in range(3):
    res = solve()
    if res and action not in solution:
        solution.append(action)
        n_solution += 1
        print(solution)
    action = 'solution %d\n'%n_solution



['solution 1\n        |        <-[R 8 ] |G B 3 5 \n8       |[R   ]->         |G B 3 5 \n8       |        <-[R G ] |B 3 5   \nG 8     |[R   ]->         |B 3 5   \nG 8     |        <-[R 5 ] |B 3     \nG 5 8   |[R   ]->         |B 3     \nG 5 8   |        <-[R B ] |3       \nR B 5 8 |[G   ]->         |3       \n']
['solution 1\n        |        <-[R 8 ] |G B 3 5 \n8       |[R   ]->         |G B 3 5 \n8       |        <-[R G ] |B 3 5   \nG 8     |[R   ]->         |B 3 5   \nG 8     |        <-[R 5 ] |B 3     \nG 5 8   |[R   ]->         |B 3     \nG 5 8   |        <-[R B ] |3       \nR B 5 8 |[G   ]->         |3       \n', 'solution 2\n        |        <-[G 5 ] |R B 3 8 \n5       |[G   ]->         |R B 3 8 \n5       |        <-[R G ] |B 3 8   \nR 5     |[G G ]->         |B 3 8   \nR 5     |        <-[G 8 ] |G B 3   \nR 5 8   |[G   ]->         |G B 3   \nR 5 8   |        <-[G B ] |G 3     \nR B 5 8 |[G   ]->         |G 3     \n']
['solution 1\n        |        <-[R 8 ] |G B 3 5 \n8       |[R

In [573]:
with open('boat.txt', 'w') as f:
    for i in solution:
        f.write(i)
        f.write('\n\n')

